# LangGraph
### StateGraph
> 복잡한 작업 흐름을 상태와 전이로 모델링해 유연하고 제어가능한 시스템 구축

**State (상태)**: 애플리케이션의 현재 스냅샷을 나타내는 공유 데이터 구조
- 특징: 주로 TypedDict나 Pydantic BaseModel을 사용- 시스템의 전체 컨텍스트를 포함

**Node (노드)**: Agent의 로직을 인코딩하는 Python 함수
- 특징    
    - 현재 State를 입력으로 받고,
    
    - 계산이나 부작용(side-effect)을 수행한 다음,
    
    - 업데이트 된 State를 반환

**Edge (엣지)** : 현재 State를 기반으로 다음에 실행할 Node를 결정하는 함수
- 특징
    - 조건부 분기 or 고정 전이
    - 시스템의 흐름을 제어


## 기능 (함수)
1. Branching: 현재 상태나 입력에 따라 다음 노드를 동적으로 결정

2. Reducer: 상태 업데이트를 관리하는 함수, 그래프의 각 노드의 출력을 그래프의 전체 상태에 통합하는 방법을 정의함.
    - Reducer X : 기존에 있던 키의 값을 덮어쓰는 방식으로 동작 (override)
    - Reducer O : 기존에 있던 값에 add를 통해 추가
    - Custom Reducer 가능

3. MessageGraph: 대화 기록을 효과적으로 관리하고 활용

4. ReAct (Resoning + Acting): 추론과 행동의 반복으로 복잡한 단계 해결

5. MemorySaver: 대화를 연속적이게 할 수 있게 해주는 것

6. Adaptive RAG: 질문의 복잡성에 따라 가장 적합한 검색 및 생성 전략을 동적으로 선택


### MessageGraph
- ChatModel을 위한 특수한 형태의 State Graph

- Messages State 정의
    1. 대화 기록을 그래프 상태에 메시지 목록으로 저장
    2. Message 객체 목록을 저장하는 messages 키를 추가
    3. 이 키에 리듀서 함수를 추가하여 메시지 업데이트를 관리
        - operator.add: 새 메시지를 기존 목록에 단순히 추가
        - add_messages 함수:기존 메시지 업데이트 처리(메시지 ID를 추적).


## LLM + Tool 연결방법
1. bind_tools() 사용
2. Tool Node 사용
    1. 최신 AIMessage의 tool_calls필드에서 도구 호출 정보 추출
    2. 추출된 도구 호출 요청들을 동시에(병렬로) 실행
    3. 각 도구 호출의 결과에 대해 ToolMessage를 생성 (결과 포함)
    4. ToolMessage는 다시 AI 모델에게 전달  -> 답변 생성

## ReAct(Reasoning + Acting)

- 정의
    
    ReAct는 '추론(Reasoning)'과 '행동(Acting)'을 결합한 접근 방식
    
- 목적
    
    LLM이 단순히 텍스트를 생성하는 것을 넘어, 환경과 상호작용 하며 복잡한 작업을 수행
    
- ReAct(Reasoning + Acting) 동작 방식
1. 행동 (Act):
- 모델이 주어진 상황을 분석하고 적절한 도구를 선택
- 선택한 도구를 호출하고 필요한 입력을 제공
2. 관찰 (Observe):
- 호출된 도구의 실행 결과나 출력을 모델에 다시 전달
- 에이전트가 자신의 행동 결과를 이해/학습
3. 추론 (Reason):
- 이전 단계의 관찰 결과를 분석하여 다음 행동을 결정
- 다른 도구를 호출하거나, 또는 직접 응답을 생성
- 현재 상황을 평가하고 최선의 다음 단계를 선택

→ 추론과 행동의 반복적인 사이클을 통해 복잡한 작업을 단계적으로 해결

1. create_react_agent() 함수로 Agent 생성
- 주요 단계:
    1. 필요한 라이브러리 임포트
    2. 언어 모델 (LLM) 설정
    3. 도구 정의
    4.  ReAct에이전트 생성
    5. 에이전트 실행

2. StateGraph 구조 사용해 Agent 생성
- 조건부 엣지 함수를 정의하면 ReAct 에이전트의 동작을 더 세밀하게 제어 가능
- 도구 호출 여부에 따라 실행 지속 여부를 결정

### MemorySaver
- 정의:
    - 그래프의 각 단계 실행 후 자동으로 상태를 저장 (체크포인터 역할)
- 목적:
    - 상태의 일시성(stateless) 문제 해결 (즉, 그래프는 각 실행마다 새로운 상태로 초기화되는 문제)
- 필요성
    - 대화의 연속성 (멀티 턴), 대화 중단 후 복원 가능, 독립적인 대화 스레드 관리

#### 체크 포인터 사용 방법

1. 메모리 사용 시 thread_id를 지정
2. 체크포인터는 그래프의 각 단계에서 상태를 기록 (모든 상태를 저장)
3. 나중에 thread_id를 사용하여 이 스레드에 접근 가능